In [44]:
!pip install scikit-learn
from sklearn.feature_extraction.text import CountVectorizer


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\pourt\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Vectoriasation des textes du corpus à l'aide d'une approche "bag of words" 


In [46]:
def vectorisation_bag_of_words(corpus):
    """
    Vectorise un corpus de texte avec l'approche Bag of Words.

    Args:
    - corpus (list): Une liste de chaînes de texte.

    Returns:
    - tuple: (Matrice BoW, Liste des mots)
    """
    # Initialiser le CountVectorizer
    vectoriseur = CountVectorizer()

    # Appliquer le CountVectorizer sur le corpus
    matrice_bow = vectoriseur.fit_transform(corpus)

    # Liste des mots (caractéristiques)
    mots = vectoriseur.get_feature_names_out()

    return matrice_bow, mots

# Regrouper les textes d'un corpus en cluster


In [8]:
import pandas as pd
df_questions = pd.read_csv('C:\\Users\\pourt\\PythonPourLaDataScience\\Datascrapping\\data.csv', encoding='latin-1', sep='|')

liste=[]
for i in range(len(df_questions['question'])):
    x=type(df_questions['question'][i])
    if x!=str:
        liste.append((x,i))
print(liste)

print(len(df_questions['question']))

df_questions=df_questions.drop(159)

[]
1308


In [53]:
matrice_bow, mots = vectorisation_bag_of_words(df_questions['question'])

In [54]:
print(matrice_bow)

  (0, 3546)	1
  (0, 11330)	1
  (0, 9495)	1
  (0, 10044)	4
  (0, 10884)	1
  (0, 5060)	14
  (0, 1282)	1
  (0, 6941)	10
  (0, 9940)	10
  (0, 15973)	1
  (0, 1403)	1
  (0, 16295)	3
  (0, 16079)	1
  (0, 12189)	1
  (0, 5136)	9
  (0, 12265)	7
  (0, 3918)	1
  (0, 1829)	1
  (0, 3500)	4
  (0, 2653)	2
  (0, 1326)	2
  (0, 13786)	4
  (0, 10069)	14
  (0, 5415)	2
  (0, 10913)	1
  :	:
  (1306, 14602)	2
  (1306, 4247)	1
  (1306, 13616)	1
  (1306, 4845)	1
  (1306, 14084)	3
  (1306, 7060)	1
  (1306, 7493)	1
  (1306, 13748)	1
  (1306, 10008)	1
  (1306, 8047)	2
  (1306, 15381)	1
  (1306, 8248)	1
  (1306, 6351)	1
  (1306, 7605)	5
  (1306, 4975)	2
  (1306, 7042)	1
  (1306, 13575)	1
  (1306, 5572)	1
  (1306, 9939)	1
  (1306, 659)	1
  (1306, 9577)	1
  (1306, 10812)	2
  (1306, 7909)	1
  (1306, 10007)	1
  (1306, 7050)	1


In [49]:
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix

def clustering_kmeans_lignes(matrice_sparse, nombre_clusters):
    """
    Effectue le clustering des lignes d'une matrice sparse avec la méthode des k-means.

    Args:
    - matrice_sparse (scipy.sparse.csr_matrix): Matrice sparse.
    - nombre_clusters (int): Nombre de clusters à former.

    Returns:
    - numpy.ndarray: Tableau des étiquettes de cluster pour chaque ligne.
    """
    if not isinstance(matrice_sparse, csr_matrix):
        raise ValueError("La matrice doit être au format csr_matrix.")


    # Appliquer la méthode des k-means
    kmeans = KMeans(n_clusters=nombre_clusters, random_state=42)
    etiquettes_clusters = kmeans.fit_predict(matrice_sparse)

    return etiquettes_clusters

# Analyse en composante principale et représentation graphique

# Illustration et test de nos fonction sur un corpus très simple 

In [50]:
corpus = ["Ceci est le premier document.", "Ceci est le deuxième document.", "Et voici le troisième document.", "Nous sommes allés en soirée Lundi.","Nous sommes allés en soirée Mardi.","Nous sommes allés en soirée Mercredi."]
#On crée un corpus de texte très simple avec 6 texte qu'on classe intuitivement en deux groupes très homogènes.

In [51]:
matrice_bow, mots = vectorisation_bag_of_words(corpus)
print("On obtient la matrice suivante :")
print(matrice_bow.toarray())
print(mots)


On obtient la matrice suivante :
[[0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0]
 [0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1]
 [1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0]
 [1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0]
 [1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0]]
['allés' 'ceci' 'deuxième' 'document' 'en' 'est' 'et' 'le' 'lundi' 'mardi'
 'mercredi' 'nous' 'premier' 'soirée' 'sommes' 'troisième' 'voici']


In [52]:
etiquettes_clusters_lignes = clustering_kmeans_lignes(matrice_bow, 2)
print("Étiquettes des clusters pour les différents texte du corpus :", etiquettes_clusters_lignes)

Étiquettes des clusters pour les différents texte du corpus : [0 0 0 1 1 1]
